# Model Setup and Testing
Test loading and inference with open-source models

In [ ]:
# Install dependencies
!pip install transformers torch accelerate huggingface_hub sentencepiece -q

In [ ]:
# Check GPU availability
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")
print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9 if torch.cuda.is_available() else 0:.2f} GB")

In [ ]:
# Login to HuggingFace
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Test model loading - StarCoder2-15B
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "bigcode/starcoder2-15b"
print(f"Loading {model_name}...")

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

print(f"✓ Model loaded successfully")
print(f"Model size: {sum(p.numel() for p in model.parameters()) / 1e9:.2f}B parameters")

In [ ]:
# Test inference
test_prompt = '''# Bug: NaN in "equals" methods
## Description
In "MathUtils", some "equals" methods will return true if both arguments are NaN.

## Reproduction
>Provide a self-contained example that reproduces this issue.
```java
public void test'''

inputs = tokenizer(test_prompt, return_tensors="pt").to(model.device)

print("Generating test...")
outputs = model.generate(
    **inputs,
    max_new_tokens=256,
    temperature=0.7,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id,
    stop_strings=["```"],
    tokenizer=tokenizer
)

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("\n" + "="*80)
print("Generated:")
print("="*80)
print(generated_text[len(test_prompt):])

In [ ]:
# Save model to cache for faster reloading
cache_dir = "/content/drive/MyDrive/libro_replication/model_cache"
model.save_pretrained(cache_dir)
tokenizer.save_pretrained(cache_dir)
print(f"✓ Model cached to {cache_dir}")

## Testing Alternative Models
Test with smaller models for ablation studies

In [ ]:
# Test with DeepSeek-Coder-7B (smaller, faster)
model_name_7b = "deepseek-ai/deepseek-coder-7b-base-v1.5"
print(f"Loading {model_name_7b}...")

tokenizer_7b = AutoTokenizer.from_pretrained(model_name_7b)
model_7b = AutoModelForCausalLM.from_pretrained(
    model_name_7b,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

print(f"✓ 7B model loaded successfully")

In [ ]:
# Test inference with 7B model
inputs_7b = tokenizer_7b(test_prompt, return_tensors="pt").to(model_7b.device)

print("Generating with 7B model...")
outputs_7b = model_7b.generate(
    **inputs_7b,
    max_new_tokens=256,
    temperature=0.7,
    do_sample=True,
    pad_token_id=tokenizer_7b.eos_token_id,
    stop_strings=["```"],
    tokenizer=tokenizer_7b
)

generated_text_7b = tokenizer_7b.decode(outputs_7b[0], skip_special_tokens=True)
print("\n" + "="*80)
print("7B Model Generated:")
print("="*80)
print(generated_text_7b[len(test_prompt):])

## Mount Google Drive for Persistence
Mount Drive to save results across Colab sessions

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Create project directory
import os
project_dir = '/content/drive/MyDrive/libro_replication'
os.makedirs(project_dir, exist_ok=True)
os.makedirs(f"{project_dir}/results", exist_ok=True)
os.makedirs(f"{project_dir}/model_cache", exist_ok=True)
print(f"✓ Project directory ready: {project_dir}")